BLEU(Bilingual Evaluation Understudy)
ROUGE(Recall-Oriented Understudy for Gisting Evaluation)

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import jieba
from typing import List

In [36]:
# 参考文本
reference_text = "近年来，随着人工智能技术的不断发展，深度学习和自然语言处理逐渐成为热门的研究领域，特别是在文本生成方面，GPT（Generative Pre-trained Transformer）模型表现出色，能够生成具有连贯性和逻辑性的长文本。GPT模型基于大规模的文本进行预训练，利用自注意力机制在生成过程中关注上下文信息，从而在各种应用场景中取得了显著的成果。例如，在智能客服系统中，GPT模型可以根据用户的问题生成合理的回答，提高了客服效率。此外，GPT模型在内容创作领域也展现出巨大的潜力，能够帮助创作者提供灵感，甚至生成整篇文章。"
generated_text = "随着人工智能的发展，深度学习和自然语言处理成为研究的热点。GPT模型在文本生成任务中表现优异，能够生成连贯的长文本。GPT模型通过自注意力机制，利用上下文信息生成复合逻辑的文本，并在诸多领域取得显著成果。例如，智能客服系统利用GPT模型生成合理的回答，大大提高了服务效率。在内容创作中，GPT模型帮助创作者提供灵感，生成初步的内容。这种能力在未来将进一步推动人工智能在各个领域的应用。"

In [37]:
# BLEU分数计算
def calculate_bleu(generated: str, reference: str):
    ref_tokens = list(jieba.cut(reference))
    gen_tokens = list(jieba.cut(generated))
    print("Reference tokens:", ref_tokens)
    print("Generated tokens:", gen_tokens)
    print("Lengths - ref:", len(ref_tokens), "gen:", len(gen_tokens))
    bleu_score = sentence_bleu(
        [ref_tokens], gen_tokens,
        smoothing_function=SmoothingFunction().method1
    )
    return bleu_score

In [38]:
# ROUGE分数计算
def calculate_rouge(generated: str, reference: str) -> dict:
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

In [39]:
bleu_score = calculate_bleu(generated_text, reference_text)
bleu_score

Reference tokens: ['近年来', '，', '随着', '人工智能', '技术', '的', '不断', '发展', '，', '深度', '学习', '和', '自然语言', '处理', '逐渐', '成为', '热门', '的', '研究', '领域', '，', '特别', '是', '在', '文本', '生成', '方面', '，', 'GPT', '（', 'Generative', ' ', 'Pre', '-', 'trained', ' ', 'Transformer', '）', '模型', '表现出色', '，', '能够', '生成', '具有', '连贯性', '和', '逻辑性', '的', '长', '文本', '。', 'GPT', '模型', '基于', '大规模', '的', '文本', '进行', '预', '训练', '，', '利用', '自', '注意力', '机制', '在', '生成', '过程', '中', '关注', '上下文', '信息', '，', '从而', '在', '各种', '应用', '场景', '中', '取得', '了', '显著', '的', '成果', '。', '例如', '，', '在', '智能', '客服', '系统', '中', '，', 'GPT', '模型', '可以', '根据', '用户', '的', '问题', '生成', '合理', '的', '回答', '，', '提高', '了', '客服', '效率', '。', '此外', '，', 'GPT', '模型', '在', '内容', '创作', '领域', '也', '展现出', '巨大', '的', '潜力', '，', '能够', '帮助', '创作者', '提供', '灵感', '，', '甚至', '生成', '整篇文章', '。']
Generated tokens: ['随着', '人工智能', '的', '发展', '，', '深度', '学习', '和', '自然语言', '处理', '成为', '研究', '的', '热点', '。', 'GPT', '模型', '在', '文本', '生成', '任务', '中', '表现', '优异', '，', '能够', '生成', '连贯

0.23562135755175467

In [40]:
bleu_score = calculate_bleu("你好呀哈哈哈哈哈哈哈哈哈哈哈哈", "你好呀哈哈哈哈哈哈哈哈哈哈哈哈")
bleu_score

Reference tokens: ['你好', '呀', '哈哈哈', '哈哈哈', '哈哈哈', '哈哈哈']
Generated tokens: ['你好', '呀', '哈哈哈', '哈哈哈', '哈哈哈', '哈哈哈']
Lengths - ref: 6 gen: 6


1.0

In [41]:
bleu_score = calculate_bleu("你好呀", "你好呀")
bleu_score

Reference tokens: ['你好', '呀']
Generated tokens: ['你好', '呀']
Lengths - ref: 2 gen: 2


0.316227766016838

In [42]:
rouge_score = calculate_rouge(generated_text, reference_text)
rouge_score

{'rouge1': Score(precision=1.0, recall=0.5, fmeasure=0.6666666666666666),
 'rouge2': Score(precision=0.6666666666666666, recall=0.2857142857142857, fmeasure=0.4),
 'rougeL': Score(precision=1.0, recall=0.5, fmeasure=0.6666666666666666)}

困惑度基于模型的预测概率分布，表示模型对下一个词的预测难度。
- 困惑度越低，代表模型对生成任务的适应性越好，生成的文本越连贯。
- 困惑度的计算通常通过模型输出的交叉墒损失计算，即计算预测词分布与真实标签之间的误差，再对损失值取对数。

基于困惑度的优化过程中应注意的问题：
- 困惑度越低，不总意味着文本质量高；困惑度衡量的是模型对文本的拟合程度，非生成内容的可读性或逻辑性。
- 设置合理的学习率。可使用动态学习率。
- 微调过程中防止过拟合。
- 数据集质量要高，噪声多的数据集会影响模型的训练过程，导致困惑度偏高。



In [43]:
# 假设的GPT模型，用于困惑度计算
class SimpleGPTModel(nn.Module):
    def __init__(self, voca_size, embedding_size):
        super(SimpleGPTModel, self).__init__()
        self.embedding = nn.Embedding(voca_size, embedding_dim=embedding_size)
        self.fc = nn.Linear(embedding_size, voca_size)
    
    def forward(self, input_ids):
        output = self.embedding(input_ids)
        output = self.fc(output)
        return output

In [44]:
# 困惑度计算
class GPTPerplexityCalculator(nn.Module):
    def __init__(self, model, vocab_size):
        super(GPTPerplexityCalculator, self).__init__()
        self.model = model
        self.vocab_size = vocab_size
    
    def forward(self, input_ids):
        output = self.model(input_ids)
        logits = output.view(-1, self.vocab_size)
        shift_labels = input_ids[:, 1:].contiguous().view(-1)
        loss = F.cross_entropy(logits[:-1], shift_labels)
        perplexity = torch.exp(loss)
        return perplexity

In [45]:
# 定义模型和假设输入
vocab_size = 300
embedding_size = 512
model = SimpleGPTModel(voca_size=vocab_size, embedding_size=embedding_size)

In [46]:
# 假设的输入
input_data = torch.randint(0, vocab_size, (1, 100))
# 计算困惑度
perplexity_calculator = GPTPerplexityCalculator(model=model, vocab_size=vocab_size)
perplexity = perplexity_calculator(input_data)
perplexity

tensor(371.7115, grad_fn=<ExpBackward0>)